In [8]:
for str in ["Python", "Jupyter", "Spark"]:
    print(f"Hello {str}")

Hello Python
Hello Jupyter
Hello Spark


# Quickstart: DataFrame
## [Grouping Data](https://spark.apache.org/docs/latest/api/python/getting_started/quickstart_df.html#Grouping-Data)   
PySpark DataFrame also provides a way of handling grouped data by using the common approach, split-apply-combine strategy. It groups the data by a certain condition applies a function to each group and then combines them back to the DataFrame.

In [11]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
df = spark.createDataFrame([
    ['red', 'banana', 1, 10], ['blue', 'banana', 2, 20], ['red', 'carrot', 3, 30],
    ['blue', 'grape', 4, 40], ['red', 'carrot', 5, 50], ['black', 'carrot', 6, 60],
    ['red', 'banana', 7, 70], ['red', 'grape', 8, 80]], schema=['color', 'fruit', 'v1', 'v2'])
df.show()

+-----+------+---+---+
|color| fruit| v1| v2|
+-----+------+---+---+
|  red|banana|  1| 10|
| blue|banana|  2| 20|
|  red|carrot|  3| 30|
| blue| grape|  4| 40|
|  red|carrot|  5| 50|
|black|carrot|  6| 60|
|  red|banana|  7| 70|
|  red| grape|  8| 80|
+-----+------+---+---+



Grouping and then applying the `avg()` function to the resulting groups.

In [9]:
df.groupby('color').avg().show()

+-----+-------+-------+
|color|avg(v1)|avg(v2)|
+-----+-------+-------+
|  red|    4.8|   48.0|
| blue|    3.0|   30.0|
|black|    6.0|   60.0|
+-----+-------+-------+



You can also apply a Python native function against each group by using pandas API.

In [10]:
def plus_mean(pandas_df):
    return pandas_df.assign(v1=pandas_df.v1 - pandas_df.v1.mean())

df.groupby('color').applyInPandas(plus_mean, schema=df.schema).show()

+-----+------+---+---+
|color| fruit| v1| v2|
+-----+------+---+---+
|black|carrot|  0| 60|
| blue|banana| -1| 20|
| blue| grape|  1| 40|
|  red|banana| -3| 10|
|  red|carrot| -1| 30|
|  red|carrot|  0| 50|
|  red|banana|  2| 70|
|  red| grape|  3| 80|
+-----+------+---+---+



## [Getting Data In/Out](https://spark.apache.org/docs/latest/api/python/getting_started/quickstart_df.html#Grouping-Data)  
CSV is straightforward and easy to use. Parquet and ORC are efficient and compact file formats to read and write faster.

In [79]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
df = spark.read.parquet('data/date=2009-01-03/part-00000-bdd84ab2-82e9-4a79-8212-7accd76815e8-c000.snappy.parquet')

df.show(truncate=False)

+----------------------------------------------------------------+-------+----+----------------------------------------------------------------+------------+------------+---------+-----------+------------+-----------+------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------+----------+
|hash                                                            |version|size|block_hash                                                      |block_number|virtual_size|lock_time|input_count|output_count|is_coinbase|output_value|outputs                                                                                                                                                                                                      

### Load Multiple Parquet Files.

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
df = spark.read.parquet("data/*")
df.show(3, truncate=False)

+----------------------------------------------------------------+-------+----+----------------------------------------------------------------+------------+-----+------------+---------+-----------+------------+-----------+-------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------+-----+-------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [5]:
df.select("hash", "block_timestamp", "date", "inputs", "outputs").printSchema()

root
 |-- hash: string (nullable = true)
 |-- block_timestamp: timestamp (nullable = true)
 |-- date: string (nullable = true)
 |-- inputs: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- address: string (nullable = true)
 |    |    |-- index: long (nullable = true)
 |    |    |-- required_signatures: long (nullable = true)
 |    |    |-- script_asm: string (nullable = true)
 |    |    |-- script_hex: string (nullable = true)
 |    |    |-- sequence: long (nullable = true)
 |    |    |-- spent_output_index: long (nullable = true)
 |    |    |-- spent_transaction_hash: string (nullable = true)
 |    |    |-- type: string (nullable = true)
 |    |    |-- value: double (nullable = true)
 |-- outputs: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- address: string (nullable = true)
 |    |    |-- index: long (nullable = true)
 |    |    |-- required_signatures: long (nullable = true)
 |    |    |-- script_asm: st

In [19]:
from pyspark.sql.functions import explode, col, format_number, lit

# Extract sender addresses from inputs
inputs_df = df.select(
    col("hash"),
    col("block_timestamp"),
    explode("inputs").alias("input")
).select(
    col("hash"),
    col("block_timestamp"),
    col("input.address").alias("address"),
    col("input.value").alias("amount"),
).withColumn("direction", lit("input"))

# Extract recipient addresses and amounts from outputs
outputs_df = df.select(
    col("hash"),
    col("block_timestamp"),
    explode("outputs").alias("output")
).select(
    col("hash"),
    col("block_timestamp"),
    col("output.address").alias("address"),
    col("output.value").alias("amount")
).withColumn("direction", lit("output"))

# Combine inputs_df and outputs_df
tx_df = inputs_df.union(outputs_df)
# Select necessary columns
tx_df = tx_df.select("hash", "block_timestamp", "direction", "address", "amount")
# Order by hash and position
tx_df = tx_df.orderBy("hash", "direction")
# Format the amount
tx_df = tx_df.withColumn("amount", format_number("amount", 8))

# Display the result
l = ["4dd37c9aa2d9c3faeedd1548c6b3edc75749a60e863cb8a14d8f561f1274b65d", "e9cb6b508ccb4ed1302b4ede21caeb6a7ceeda7f399d5e920160a462155af430", "ca9388fb4b6d2f92c95f1713b99b9f250d6f5635843a2fce9b2417924b2e0297"]
tx_df.filter(col("hash").isin(l)).show(truncate=False)

+----------------------------------------------------------------+-------------------+---------+----------------------------------+-----------+
|hash                                                            |block_timestamp    |direction|address                           |amount     |
+----------------------------------------------------------------+-------------------+---------+----------------------------------+-----------+
|4dd37c9aa2d9c3faeedd1548c6b3edc75749a60e863cb8a14d8f561f1274b65d|2011-10-02 02:18:55|input    |17TLtRVBZKtWzErFwisMjwnW1RtFUx5Wou|0.01637169 |
|4dd37c9aa2d9c3faeedd1548c6b3edc75749a60e863cb8a14d8f561f1274b65d|2011-10-02 02:18:55|input    |1GdsUE2KuVXVuKPXwGYVYX2cwuX1cyvTkj|0.04005355 |
|4dd37c9aa2d9c3faeedd1548c6b3edc75749a60e863cb8a14d8f561f1274b65d|2011-10-02 02:18:55|input    |184Gxrr6pm7ScU2pSB8hibGwW2GttBpY3m|0.49950000 |
|4dd37c9aa2d9c3faeedd1548c6b3edc75749a60e863cb8a14d8f561f1274b65d|2011-10-02 02:18:55|input    |1HVe3xJ9iG8fjqd5RcNA2aJsiGoskApUth|3.300